In [19]:
# Import dependencies
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.activations import tanh
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
tokenizer.fit_on_texts([shakespeare_text])

In [4]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [5]:
print(dataset_size)

1


In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [7]:
shape = encoded.shape[0]

In [8]:
train_size = shape * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [9]:
train_size

1003854

In [10]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift = 1, drop_remainder = True)

In [11]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [12]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [13]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [14]:
# Reminder that categorical features should be one-hot encoded
# This is what we do to our flat mapped dataset
dataset = dataset.map(lambda X_batch, Y_batch : (tf.one_hot(X_batch, depth = max_id), Y_batch))

# Finally add prefetching
dataset = dataset.prefetch(1)

In [33]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences = True, input_shape = [None, max_id], dropout = 0.2, 
                     activation = tanh),
    keras.layers.GRU(128, return_sequences = True, dropout = 0.2, 
                     activation = tanh),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation = 'softmax'))
])

In [34]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')

In [36]:
model.fit(dataset, epochs = 5)

Epoch 1/5
31368/31368 [==============================] - 273s 9ms/step - loss: 1.6152
Epoch 2/5
31368/31368 [==============================] - 269s 9ms/step - loss: 1.5358
Epoch 3/5
31368/31368 [==============================] - 266s 8ms/step - loss: 1.5127
Epoch 4/5
31368/31368 [==============================] - 269s 9ms/step - loss: 1.5014
Epoch 5/5
31368/31368 [==============================] - 273s 9ms/step - loss: 1.4935


In [37]:
def preprocess(strings):
    # Tokenize each string into a sequence of individual characters
    sequences = []
    for string in strings:
        sequence = []
        for char in string:
            sequence.append(char)
        sequences.append(sequence)
    # Pad the sequences to a fixed length
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')
    return padded_sequences

In [38]:
def generate_letter(model, tokenizer, sequence):
    # Encode the sequence
    X_new = tokenizer.texts_to_sequences([sequence])[0]
    # Pad the sequence to a fixed length
    X_new = tf.keras.preprocessing.sequence.pad_sequences([X_new], maxlen=50, padding='post')
    # Predict the probability distribution over the vocabulary of individual characters
    y_proba = model.predict(X_new)[0]
    # Sample from the distribution to get the next letter index
    letter_index = np.random.choice(len(y_proba), p=y_proba)
    # Convert the index to a letter and return it
    return tokenizer.index_word[letter_index]

In [39]:
sequence = 'How are yo'

while True:
    next_letter = generate_letter(model, tokenizer, sequence)
    sequence += next_letter
    if next_letter == '.':
        break
        
print(sequence)

ValueError: in user code:

    File "C:\Users\quinl\anaconda3\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\quinl\anaconda3\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\quinl\anaconda3\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\quinl\anaconda3\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Users\quinl\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\quinl\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 232, in assert_input_compatibility
        raise ValueError(

    ValueError: Exc

In [ ]:
def next_char(text, temperature = 1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars = 50, temperature = 1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text